# Stable Diffusion WebUI Colab TW

### 使用流程

1. 到**前置步驟**, **下載模型**, **Extensions**, **其他設置**進行設置
2. 左上角工具列 > 執行階段 > 變更執行階段類型 > 硬體加速器改為 GPU
3. 左上角工具列 > 執行階段 > 全部執行
4. 到**啟動 WebUI** 點選 public URL

### 巴哈原文
https://home.gamer.com.tw/artwork.php?sn=5664550

### 參考資料
https://github.com/ddPn08/automatic1111-colab

### 更新時間
2023/04/20

## 1 - 前置步驟

In [ ]:
# @title 1.1 下載 stable-diffusion-webui
%cd /content
webui_version = "vladmandic AUTOMATIC1111(\u6700\u65B0\u7248\u672C)" # @param ["vladmandic AUTOMATIC1111(最新版本)", "vladmandic AUTOMATIC1111(05/06)", "vladmandic AUTOMATIC1111(directml-stable)", "vladmandic AUTOMATIC1111(diffusers)", "其他(填寫repository_url和webui_branch)"]
repository_url = ""  # @param {type: "string"}
webui_branch = ""  # @param {type: "string"}
# @markdown ---
# @markdown - https://github.com/vladmandic/automatic/tree/master
# @markdown - https://github.com/CLI1976/vladmandic_automatic-

if webui_version == "vladmandic AUTOMATIC1111(最新版本)":
    repository_url = "https://github.com/vladmandic/automatic"
    webui_branch = "master"
elif webui_version == "vladmandic AUTOMATIC1111(05/06)":
    repository_url = "https://github.com/CLI1976/vladmandic_automatic-"
    webui_branch = "master"
elif webui_version == "vladmandic AUTOMATIC1111(directml-stable)":
    repository_url = "https://github.com/vladmandic/automatic"
    webui_branch = "directml-stable"
elif webui_version == "vladmandic AUTOMATIC1111(diffusers)":
    repository_url = "https://github.com/vladmandic/automatic"
    webui_branch = "diffusers"  

! git clone {repository_url} stable-diffusion-webui

%cd /content/stable-diffusion-webui
! git checkout -f {webui_branch}



/content
Cloning into 'stable-diffusion-webui'...
remote: Enumerating objects: 22992, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 22992 (delta 111), reused 146 (delta 81), pack-reused 22781
Receiving objects: 100% (22992/22992), 32.15 MiB | 3.16 MiB/s, done.
Resolving deltas: 100% (16164/16164), done.
/content/stable-diffusion-webui
Already on 'master'
Your branch is up to date with 'origin/master'.


In [ ]:
# @title 1.2 建立雲端資料夾
%cd /content
# @markdown 雲端資料夾名稱
data_dir_name = "vladmandic-webui"  # @param {type:"string"}
data_dir = f"/content/drive/MyDrive/{data_dir_name}"
# @markdown 不連接 models/Stable-diffusion 資料夾模式
sd_no_link_mode = False # @param {type:"boolean"}

import os

drive_path = "/content/drive"
from google.colab import drive

try:
    drive.mount(drive_path, force_remount=True)
    os.makedirs(data_dir, exist_ok=True)
except:
    print('Mount Error')
    raise

if os.path.exists(f"{data_dir}/script.pre.sh"):
    ! chmod +x {data_dir}/script.pre.sh
    ! {data_dir}/script.pre.sh

models_path = f"{data_dir}/models"
output_path = f"{data_dir}/outputs"
config_path = f"{data_dir}/config"
scripts_path = f"{data_dir}/scripts"
extensions_file_path = f"{data_dir}/extensions.txt"

os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)
os.makedirs(config_path, exist_ok=True)
os.makedirs(scripts_path, exist_ok=True)
os.makedirs(f"{models_path}/Stable-diffusion", exist_ok=True)
os.makedirs(f"{models_path}/VAE", exist_ok=True)
os.makedirs(f"{models_path}/embeddings", exist_ok=True)
os.makedirs(f"{models_path}/hypernetworks", exist_ok=True)
os.makedirs(f"{models_path}/Lora", exist_ok=True)
os.makedirs(f"{models_path}/ControlNet", exist_ok=True)

for script in os.listdir(scripts_path):
    if not os.path.isfile(f"stable-diffusion-webui/scripts/{script}") and os.path.isfile(f"{scripts_path}/{script}"):
        ! rm -f stable-diffusion-webui/scripts/{script} && ln -s {scripts_path}/{script} stable-diffusion-webui/scripts/{script}

for dir in os.listdir(models_path):
    if dir == "embeddings":
        ! rm -Rf stable-diffusion-webui/embeddings && ln -s {models_path}/embeddings stable-diffusion-webui/embeddings
    elif dir != "ControlNet":
        if dir == "Stable-diffusion" and sd_no_link_mode == True:
            ! rm -Rf stable-diffusion-webui/models/Stable-diffusion && mkdir stable-diffusion-webui/models/Stable-diffusion
            continue   
        ! rm -Rf stable-diffusion-webui/models/{dir} && ln -s {models_path}/{dir} stable-diffusion-webui/models/{dir}

! rm -Rf stable-diffusion-webui/outputs && ln -s {data_dir}/outputs stable-diffusion-webui/outputs

for filename in ["config.json", "ui-config.json", "styles.csv", "extensions.txt"]:
    ! rm -f stable-diffusion-webui/{filename}
    filepath = f"{config_path}/{filename}"
    if not os.path.exists(filepath):
        if filename.endswith(".json"):
            with open(filepath, mode="w") as f:
                f.write("{}")
        else:
            ! touch {config_path}/{filename}
    ! ln -s {config_path}/{filename} stable-diffusion-webui/{filename}

/content
Mounted at /content/drive


## 3 - Extensions

In [ ]:
# @title 3.1 下載 Extensions
locon = False # @param {type:"boolean"}
sd_dynamic_prompts = True # @param {type:"boolean"}
ControlNet_v1_1_nightly = True # @param {type:"boolean"}
ultimate_upscale_for_automatic1111 = True # @param {type:"boolean"}
sd_face_editor = True # @param {type:"boolean"}
localization_zh_TW = False # @param {type:"boolean"}
bilingual_localization = False # @param {type:"boolean"}
tag_complete = False # @param {type:"boolean"}
image_browser = False # @param {type:"boolean"}
Civitai_Helper = False # @param {type:"boolean"}
tunnels = False # @param {type:"boolean"}
cutoff = False # @param {type:"boolean"}
control_net = False # @param {type:"boolean"}
openpose_editor = False # @param {type:"boolean"}
posex = False # @param {type:"boolean"}
depth_lib = True # @param {type:"boolean"}
latent_couple = False # @param {type:"boolean"}
composable_lora = False # @param {type:"boolean"}
lora_block_weight = False # @param {type:"boolean"}
extension_urls = ""  # @param {type:"string"}

# @markdown ---
# @markdown - [[AI繪圖] Stable Diffusion WebUI Colab TW Extensions整理](https://home.gamer.com.tw/artwork.php?sn=5678466)
# @markdown - 網址之間用英文逗號隔開。例：`url1`, `url2`, `url3`
# @markdown - 如果使用WebUI內建方法下載 Extension，仍然要執行 `下載 Extensions` 以記錄下載的 Extension。
# @markdown - 如果不想再下載某個 Extension，請刪掉 `config/extensions.txt` 上對應的 GitHub 網址。

extensions_pair = [   
            (locon, "https://github.com/KohakuBlueleaf/a1111-sd-webui-locon"),
            (sd_dynamic_prompts, "https://github.com/adieyal/sd-dynamic-prompts"),
            (ControlNet_v1_1_nightly, "https://github.com/lllyasviel/ControlNet-v1-1-nightly"),
            (ultimate_upscale_for_automatic1111, "https://github.com/Coyote-A/ultimate-upscale-for-automatic1111"),
            (sd_face_editor, "https://github.com/ototadana/sd-face-editor"),
            (localization_zh_TW, "https://github.com/benlisquare/stable-diffusion-webui-localization-zh_TW"),
            (bilingual_localization, "https://github.com/journey-ad/sd-webui-bilingual-localization"),
            (tag_complete, "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete"),
            (image_browser, "https://github.com/AlUlkesh/stable-diffusion-webui-images-browser"),
            (Civitai_Helper, "https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper"),
            (tunnels, "https://github.com/camenduru/sd-webui-tunnels"),
            (cutoff, "https://github.com/hnmr293/sd-webui-cutoff"),
            (control_net, "https://github.com/Mikubill/sd-webui-controlnet"),
            (openpose_editor, "https://github.com/fkunn1326/openpose-editor"),
            (posex, "https://github.com/hnmr293/posex"),
            (depth_lib, "https://github.com/jexom/sd-webui-depth-lib"),
            (latent_couple, "https://github.com/ashen-sensored/stable-diffusion-webui-two-shot"),
            (composable_lora, "https://github.com/opparco/stable-diffusion-webui-composable-lora"),
            (lora_block_weight,"https://github.com/hako-mikan/sd-webui-lora-block-weight")
          ]

with open('/content/stable-diffusion-webui/extensions.txt', 'r') as f:
    extensions_txt = f.read()
extensions_txt = list(map(str.strip, extensions_txt.split("\n")))

extension_urls = list(map(str.strip, extension_urls.split(",")))

extensions = []
  
for name, url in extensions_pair:
    if name == True and url not in extensions:
        extensions.append(url)
for url in extension_urls:
    if url != '' and url not in extensions:
        extensions.append(url)
for url in extensions_txt:
    if url != '' and url not in extensions:
        extensions.append(url)

import os

if extensions != []:
    %cd /content/stable-diffusion-webui/extensions
    for extension in extensions:
        extension_name, _ = os.path.splitext(extension.split("/")[-1])
        if not os.path.exists(extension_name):
            ! git clone {extension}

%cd /content/stable-diffusion-webui/extensions
if os.path.exists("sd-webui-lora-block-weight") and not os.path.isfile(f"{scripts_path}/lbwpresets.txt"):
    lbwpresets = "\
NONE:0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0\n\
ALL:1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1\n\
INS:1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0\n\
IND:1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0\n\
INALL:1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0\n\
MIDD:1,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0\n\
OUTD:1,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0\n\
OUTS:1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1\n\
OUTALL:1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1\n\
ALL0.5:0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5"
    with open(f"{scripts_path}/lbwpresets.txt", mode='w', encoding="utf-8") as f:
        f.write(lbwpresets)
    ! rm -f /content/stable-diffusion-webui/scripts/lbwpresets.txt && ln -s {scripts_path}/lbwpresets.txt /content/stable-diffusion-webui/scripts/lbwpresets.txt

%cd /content/stable-diffusion-webui/extensions
! echo -n '' > /content/stable-diffusion-webui/extensions.txt
for dir in next(os.walk("/content/stable-diffusion-webui/extensions"))[1]:
    %cd {dir}
    if os.path.exists(".git"):
        ! git config --get remote.origin.url >> /content/stable-diffusion-webui/extensions.txt
    %cd .. 

/content/stable-diffusion-webui/extensions
Cloning into 'sd-dynamic-prompts'...
remote: Enumerating objects: 5008, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 5008 (delta 20), reused 30 (delta 14), pack-reused 4963
Receiving objects: 100% (5008/5008), 26.73 MiB | 12.79 MiB/s, done.
Resolving deltas: 100% (1821/1821), done.
Cloning into 'ControlNet-v1-1-nightly'...
remote: Enumerating objects: 2371, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 2371 (delta 96), reused 125 (delta 75), pack-reused 2181
Receiving objects: 100% (2371/2371), 65.18 MiB | 22.89 MiB/s, done.
Resolving deltas: 100% (966/966), done.
Cloning into 'ultimate-upscale-for-automatic1111'...
remote: Enumerating objects: 296, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 296 (delta 29), reused 25 (delta 9), pack

In [ ]:
# @title 3.3 Download files to a specified folder.(下載檔案到指定資料夾) 
target_dir = "" # @param {type:"string"} 
file_urls = "" # @param {type:"string"}
# @markdown ---
# @markdown - Fill in the `target_dir` with the specified folder, and fill in the download URLs of the files in `file_urls` 
# @markdown - 將指定資料夾填在 `target_dir`，檔案的下載網址填在 `file_urls`。
# @markdown - Separate the URLs with a comma. For example: `url1`, `url2`, `url3`.
# @markdown - 網址之間用英文逗號隔開。例：`url1`, `url2`, `url3`
file_urls = list(map(str.strip, file_urls.split(",")))

import os
if os.path.exists(os.path.split(target_dir)[0]):
    os.makedirs(target_dir, exist_ok=True)
    %cd {target_dir}
    for url in file_urls:
        if url != '':
            ! wget -nc --content-disposition {url}

In [ ]:
# @title 3.4 Custom Folder Link
google_drive_dir = "/content/drive/MyDrive/Stable_Diffusion_WebUI_Colab_TW/scripts/wildcards" # @param {type:"string"} 
webui_dir = "/content/stable-diffusion-webui/extensions/sd-dynamic-prompts/wildcards" # @param {type:"string"}
# @markdown ---
# @markdown - Fill in the cloud drive folder in `google_drive_dir` and the WebUI folder in `webui_dir`.

import os
if os.path.exists(os.path.split(webui_dir)[0]):
    os.makedirs(google_drive_dir, exist_ok=True)
    if os.path.exists(webui_dir):
        ! cp -Rf {webui_dir}/* {google_drive_dir}
    else:
        os.makedirs(webui_dir, exist_ok=True)
    ! rm -Rf {webui_dir} && ln -s {google_drive_dir} {webui_dir}

In [ ]:
# @title 3.5 Custom Folder Link
google_drive_dir = "/content/drive/MyDrive/Stable_Diffusion_WebUI_Colab_TW/extensions/sd-webui-controlnet/models" # @param {type:"string"} 
webui_dir = "/content/stable-diffusion-webui/models/sd-webui-controlnet" # @param {type:"string"}
# @markdown ---
# @markdown - Fill in the cloud drive folder in `google_drive_dir` and the WebUI folder in `webui_dir`.

import os
if os.path.exists(os.path.split(webui_dir)[0]):
    os.makedirs(google_drive_dir, exist_ok=True)
    if os.path.exists(webui_dir):
        ! cp -Rf {webui_dir}/* {google_drive_dir}
    else:
        os.makedirs(webui_dir, exist_ok=True)
    ! rm -Rf {webui_dir} && ln -s {google_drive_dir} {webui_dir}

In [ ]:
# @title 3.6 Custom Folder Link
google_drive_dir = "/content/drive/MyDrive/Stable_Diffusion_WebUI_Colab_TW/ultrasharp" # @param {type:"string"} 
webui_dir = "/content/stable-diffusion-webui/models/ESRGAN" # @param {type:"string"}
# @markdown ---
# @markdown - Fill in the cloud drive folder in `google_drive_dir` and the WebUI folder in `webui_dir`.

import os
if os.path.exists(os.path.split(webui_dir)[0]):
    os.makedirs(google_drive_dir, exist_ok=True)
    if os.path.exists(webui_dir):
        ! cp -Rf {webui_dir}/* {google_drive_dir}
    else:
        os.makedirs(webui_dir, exist_ok=True)
    ! rm -Rf {webui_dir} && ln -s {google_drive_dir} {webui_dir}

## 4 - Start WebUI

In [ ]:
# @title 4.1 Start WebUI
# @markdown - Click on the Public WebUI Colab URL to open the WebUI<br>

%cd /content/stable-diffusion-webui/
# ! python launch.py  --share
# ! python launch.py  --help

!COMMANDLINE_ARGS="--no-hashing --share --ckpt /content/stable-diffusion-webui/models/Stable-diffusion/lyriel_v15.safetensors --vae /content/stable-diffusion-webui/models/VAE/vae-ft-mse-840000-ema-pruned.safetensors" REQS_FILE="requirements.txt" python launch.py

  # @markdown -  options:
  # @markdown - -h, --help                           show this help message and exit
  # @markdown - --no-half                            Do not switch the model to 16-bit floats
  # @markdown - --no-half-vae                        Do not switch the VAE model to 16-bit floats
  # @markdown - --precision {full,autocast}          Evaluate at this precision
  # @markdown - --medvram                            Enable model optimizations for sacrificing a little speed for low memory usage
  # @markdown - --lowvram                            Enable model optimizations for sacrificing a lot of speed for lowest memory usage
  # @markdown - --lowram                             Load checkpoint weights to VRAM instead of RAM
  # @markdown - --ckpt CKPT                          Path to checkpoint of stable diffusion model to load immediately
  # @markdown - --vae VAE                            Path to checkpoint of stable diffusion VAE model to load immediately
  # @markdown - --data-dir DATA_DIR                  Base path where all user data is stored
  # @markdown - --allow-code                         Allow custom script execution
  # @markdown - --share                              Enable to make the UI accessible through Gradio site
  # @markdown - --enable-insecure                    Enable extensions tab regardless of other options
  # @markdown - --use-cpu USE_CPU [USE_CPU ...]      Force use CPU for specified modules
  # @markdown - --listen                             Launch web server using public IP address
  # @markdown - --port PORT                          Launch web server with given server port
  # @markdown - --hide-ui-dir-config                 Hide directory configuration from UI
  # @markdown - --freeze-settings                    Disable editing settings
  # @markdown - --gradio-auth GRADIO_AUTH            Set Gradio authentication like "username:password,username:password""
  # @markdown - --gradio-auth-path GRADIO_AUTH_PATH  Set Gradio authentication using file
  # @markdown - --autolaunch                         Open the UI URL in the system's default browser upon launch
  # @markdown - --disable-console-progressbars       Do not output progressbars to console
  # @markdown - --disable-safe-unpickle              Disable checking models for malicious code
  # @markdown - --api-auth API_AUTH                  Set API authentication
  # @markdown - --api-log                            Enable logging of all API requests
  # @markdown - --device-id DEVICE_ID                Select the default CUDA device to use
  # @markdown - --cors-origins CORS_ORIGINS          Allowed CORS origin(s) in the form of a comma-separated list
  # @markdown - --cors-regex CORS_REGEX              Allowed CORS origin(s) in the form of a single regular expression
  # @markdown - --tls-keyfile TLS_KEYFILE            Partially enables TLS, requires --tls-certfile to fully function
  # @markdown - --tls-certfile TLS_CERTFILE          Partially enables TLS, requires --tls-keyfile to fully function
  # @markdown - --server-name SERVER_NAME            Sets hostname of server
  # @markdown - --no-hashing                         Disable sha256 hashing of checkpoints
  # @markdown - --no-download-sd-model               Disable download of default model even if no model is found
  # @markdown -   --profile                            Run profiler, default: False
  # @markdown - --debug                              Run installer with debug logging, default: False
  # @markdown - --quick                              Skip installing if setup.log is newer than repo timestamp, default: False
  # @markdown - --upgrade                            Upgrade main repository to latest version, default: False
  # @markdown - --noupdate                           Skip update extensions and submodules, default: False
  # @markdown - --skip-extensions                    Skips running individual extension installers, default: False

Streaming output truncated to the last 5000 lines.
	Tile 6/15
	Tile 7/15
	Tile 8/15
	Tile 9/15
	Tile 10/15
	Tile 11/15
	Tile 12/15
	Tile 13/15
	Tile 14/15
	Tile 15/15
Tile size: 512x512
Tiles amount: 24
Grid: 6x4
Redraw enabled: True
Seams fix mode: NONE
Loading model from cache: control_v11f1e_sd15_tile [a371b31b]
Loading preprocessor: tile_resample
Pixel Perfect Mode Enabled.
resize_mode = ResizeMode.RESIZE
raw_H = 576
raw_W = 576
target_H = 576
target_W = 576
estimation = 576.0
preprocessor resolution = 576
100% 12/12 [00:02<00:00,  5.81it/s]
Loading model from cache: control_v11f1e_sd15_tile [a371b31b]
Loading preprocessor: tile_resample
Pixel Perfect Mode Enabled.
resize_mode = ResizeMode.RESIZE
raw_H = 576
raw_W = 576
target_H = 576
target_W = 576
estimation = 576.0
preprocessor resolution = 576
100% 12/12 [00:02<00:00,  5.87it/s]
Loading model from cache: control_v11f1e_sd15_tile [a371b31b]
Loading preprocessor: tile_resample
Pixel Perfect Mode Enabled.
resize_mode = ResizeMode.